In [1]:
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("smart_environment_data_Official.csv")


# Display the first few rows
print(df.head())

          timestamp sensor_id     co2_ppm  pm25_ugm3  temperature_c  \
0   07/05/2025 4:14   SENS762  433.043819  35.411776           30.7   
1  06/05/2025 23:48   SENS414  378.758719  19.912831           34.1   
2  06/05/2025 13:46   SENS874  411.669284  39.356963           31.2   
3   07/05/2025 2:09   SENS903  475.838638  29.126293           28.7   
4   06/05/2025 8:35   SENS114  396.233177  40.948456           32.0   

   humidity_percent  soil_moisture_percent  water_ph  turbidity_ntu  
0              64.6                   28.8      6.88           1.82  
1              83.7                   29.3      8.18           4.00  
2              85.1                   22.7      6.89           1.34  
3              76.8                   15.6      7.85           1.13  
4              81.7                   27.9      7.73           0.04  


In [2]:
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")


✅ Connected to Ganache successfully!


In [4]:
# Replace with actual contract address from Remix
contract_address = "0x1549608d05D5484c8e323a79d721c84EDb92b801"


# ABI from Remix
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": False, "internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"indexed": False, "internalType": "string", "name": "sensorId", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "MetricType", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "MetricValue", "type": "string"}
        ],
        "name": "DataStored",
        "type": "event"
    },
    {
        "inputs": [],
        "name": "MAX_ENTRIES",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "name": "dataRecords",
        "outputs": [
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "sensorId", "type": "string"},
            {"internalType": "string", "name": "MetricType", "type": "string"},
            {"internalType": "string", "name": "MetricValue", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "index", "type": "uint256"}],
        "name": "getRecord",
        "outputs": [
            {"internalType": "uint256", "name": "", "type": "uint256"},
            {"internalType": "string", "name": "", "type": "string"},
            {"internalType": "string", "name": "", "type": "string"},
            {"internalType": "string", "name": "", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getTotalRecords",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [{"internalType": "address", "name": "", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "_sensorId", "type": "string"},
            {"internalType": "string", "name": "_MetricType", "type": "string"},
            {"internalType": "string", "name": "_MetricValue", "type": "string"}
        ],
        "name": "storeData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "_sensorId", "type": "string"},
            {"internalType": "string", "name": "co2", "type": "string"},
            {"internalType": "string", "name": "pm25", "type": "string"},
            {"internalType": "string", "name": "temperature", "type": "string"},
            {"internalType": "string", "name": "humidity", "type": "string"},
            {"internalType": "string", "name": "soilMoisture", "type": "string"},
            {"internalType": "string", "name": "waterPh", "type": "string"},
            {"internalType": "string", "name": "turbidity", "type": "string"}
        ],
        "name": "storeFullSensorData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    } 
]


# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set default sender (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0x1549608d05D5484c8e323a79d721c84EDb92b801


In [13]:
import time


def send_iot_data(sensor_id, metric_type, metric_value):
    """Sends IoT data to the deployed smart contract"""
    
    print(f"📡 Sending: Sensor={sensor_id}, Type={metric_type}, Value={metric_value}")
    
    txn = contract.functions.storeData(sensor_id, metric_type, metric_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
   
    receipt = web3.eth.wait_for_transaction_receipt(txn, timeout=60)
    print(f"✅ Data Stored: {metric_type} - {metric_value}, Txn Hash: {receipt.transactionHash.hex()}")


In [15]:
metrics = ["co2_ppm", "pm25_ugm3", "temperature_c", "humidity_percent", 
           "soil_moisture_percent", "water_ph", "turbidity_ntu"]

for _, row in df.iterrows():
    sensor_id = str(row["sensor_id"])
    for metric in metrics:
        metric_value = str(row[metric])
        send_iot_data(sensor_id, metric, metric_value)
        time.sleep(1)  # prevent flooding the blockchain


📡 Sending: Sensor=SENS762, Type=co2_ppm, Value=433.0438194
✅ Data Stored: co2_ppm - 433.0438194, Txn Hash: c589f70cce824bd6103f6b27b76175d93ab676ca078dc0eda71eca8210d62c3b
📡 Sending: Sensor=SENS762, Type=pm25_ugm3, Value=35.41177588
✅ Data Stored: pm25_ugm3 - 35.41177588, Txn Hash: ba57d322fc46db9a053a96774320df600427e451545bda60ee9deaeafc603d2a
📡 Sending: Sensor=SENS762, Type=temperature_c, Value=30.7
✅ Data Stored: temperature_c - 30.7, Txn Hash: 9246e46bf56c4368e2db0f86b9193a0cf1d0f9bad5946e5a903d4a720fdc3c35
📡 Sending: Sensor=SENS762, Type=humidity_percent, Value=64.6
✅ Data Stored: humidity_percent - 64.6, Txn Hash: bf919a512d7439da28259c4c0bab57ea9140e034af2675cab2cc3818cec26d22
📡 Sending: Sensor=SENS762, Type=soil_moisture_percent, Value=28.8
✅ Data Stored: soil_moisture_percent - 28.8, Txn Hash: 76f96d7159dd9927511d5884b9444317e617af0008a4065ec0324af8dee7b0b5
📡 Sending: Sensor=SENS762, Type=water_ph, Value=6.88
✅ Data Stored: water_ph - 6.88, Txn Hash: 4c08c7ff0d2ce28c1201d75d3

📡 Sending: Sensor=SENS115, Type=turbidity_ntu, Value=4.35
✅ Data Stored: turbidity_ntu - 4.35, Txn Hash: 13581907effcf5cf1f8a92cc5c60d89fda29c1a6fdbbbc05afff76354e0b246f
📡 Sending: Sensor=SENS478, Type=co2_ppm, Value=438.8756332
✅ Data Stored: co2_ppm - 438.8756332, Txn Hash: c8b1dc096b40ddfcdd9a9b21df7a466ef9ee9b1ced35f69eec92f7f278fcf15d
📡 Sending: Sensor=SENS478, Type=pm25_ugm3, Value=47.76734479
✅ Data Stored: pm25_ugm3 - 47.76734479, Txn Hash: 56d1fcaf6f2455ed66f133c397f03ce43da8064d01138c24b7f03d066174ec9f
📡 Sending: Sensor=SENS478, Type=temperature_c, Value=33.8
✅ Data Stored: temperature_c - 33.8, Txn Hash: b378cb4b2629578880a15b6ebdf58ac1925a90a8f2ce92367ab8a76553939221
📡 Sending: Sensor=SENS478, Type=humidity_percent, Value=51.7
✅ Data Stored: humidity_percent - 51.7, Txn Hash: 41472afae1d98110b470ff0510a35dfe788927aeddb07a09b8c9b23ae3d695eb
📡 Sending: Sensor=SENS478, Type=soil_moisture_percent, Value=21.6
✅ Data Stored: soil_moisture_percent - 21.6, Txn Hash: 1615b74d68bad21

📡 Sending: Sensor=SENS153, Type=water_ph, Value=8.14
✅ Data Stored: water_ph - 8.14, Txn Hash: 21227c3ca79c5ed38deb1727adeaa800a41c723dff6da350f204db8457711faf
📡 Sending: Sensor=SENS153, Type=turbidity_ntu, Value=2.92
✅ Data Stored: turbidity_ntu - 2.92, Txn Hash: 07ff112fb59543c0809ab3836c1d617fa63373d2c8c047be92315e5f778366f1
📡 Sending: Sensor=SENS425, Type=co2_ppm, Value=424.7340591
✅ Data Stored: co2_ppm - 424.7340591, Txn Hash: ecddfb37f144afd3785cc9f754e9e1adf7612861d5d12348a725397910eb520d
📡 Sending: Sensor=SENS425, Type=pm25_ugm3, Value=21.44371855


Web3RPCError: {'message': 'VM Exception while processing transaction: revert Storage limit reached', 'stack': 'RuntimeError: VM Exception while processing transaction: revert Storage limit reached\n    at EIP1559FeeMarketTransaction.fillFromResult (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:12745)\n    at Miner.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:36703)\n    at async Miner.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:35116)\n    at async Miner.mine (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:39680)\n    at async Blockchain.mine (C:\\Program Files\\WindowsApps\\Ganach

In [16]:
# Get total stored records
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")


Total IoT records stored: 100


In [19]:
# Retrieve and print a specific record 

record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

First Stored Record: [1747997549, 'TEST001', 'Temperature', '22.5°C']
